- This is a simple inference notebook to see the outputs from the very earry tuned version of StableLM JP models.
- The template has two main components:
  - 1. instruction - This is usually an actual user input for the model.
  - 2. input - This is named `input` but an additional information that comes with the `instruction`.

In [1]:
from dataclasses import dataclass
import os

import torch
from torch.utils.data import Dataset, random_split
from tqdm.auto import tqdm

from datasets import load_dataset
import transformers as T
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

2023-05-24 09:29:00.726141: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 09:29:02.932897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
INPUT_PROMPT = """以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
{instruction}

### 入力: 
{input}

### 応答: """

NO_INPUT_PROMPT = """以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
{instruction}

### 応答: """


def postprocess_output(output):
    output = output\
        .split('### 応答:')[1]\
        .split('###')[0]\
        .split('##')[0]\
        .lstrip(tokenizer.bos_token)\
        .rstrip(tokenizer.eos_token)\
        .replace("###", "")\
        .strip()
    return output

In [4]:
@dataclass
class args:
    tokenizer_name_or_path: str = "/PATH/TO/tokenizer/"
    model_name_or_path: str = "/PATH/TO/CHECKPOINT"
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name_or_path, use_fast=False)

In [6]:
model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path)

In [8]:
instruction = "日本の首都は？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


首都は東京です！


In [8]:
instruction = "アメリカの首都は？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


ワシントンD.C.


In [9]:
instruction = "人工知能とはどのような技術ですか？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


機械学習（ML）は、コンピュータがデータからパターンや特徴を見つけ出し、予測を行うために使用されます。このプロセスは、大量のデータセットを使用して実行されますが、その多くは人間の介入を必要としません。MLの最も一般的な用途には、画像認識、自然言語処理、自動運転車などのアプリケーションがあります。


In [12]:
instruction = "次の記事を要約してください。"
input_="""【ニューヨーク=共同】米誌タイムは10日までに、岸田文雄首相を表紙に掲載した次回号（12日発売）の内容を一部公開した。「日本の選択」と題し、岸田氏が「長年の平和主義を捨て去り、自国を真の軍事大国にすることを望んでいる」と紹介した。
記事では岸田氏が19〜21日の主要7カ国首脳会議（G7広島サミット）で、中国やロシア、北朝鮮といった国々に立ち向かうため民主主義国の団結を狙うと説明。防衛費の増額で「世界3位の経済大国に見合った軍事的影響力を持つ国にしようとしている」とした。
一方で「核兵器のない世界」を目指す岸田氏の理念と防衛力強化が矛盾するとの意見があると指摘。経済的な結び付きが強い中国との向き合い方など、課題も抱えていると分析した。
エマニュエル駐日米大使はツイッターで、岸田氏が外交などで「指導力を発揮している」と述べ、表紙を飾ったことに祝意を示した。岸田氏は4月28日に首相公邸でタイムの取材に応じた。"""
input_text = INPUT_PROMPT.format(instruction=instruction, input=input_)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=512,
        temperature=0.4,
        do_sample=True,
        no_repeat_ngram_size=3,
        # repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


ニューヨーク・タイムズ紙は、岸田が「日本の選択（10日付）」と題する記事で、中国、ロシア、そして北朝鮮に対抗するため、民主主義国の一員として立ち向かうと説明していることを報じた。


In [13]:
instruction = "あなたはAIアシスタントです。AIアシスタントとしてふるまってください。そのうえで自己紹介をお願いします。"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


私はAIアシスタントのAlexaです。よろしくお願いします！


In [14]:
instruction = "AIについてどう思いますか？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


私はAIが人間よりも優れているとは考えていません。しかし、AIの進歩により多くのことが可能になることは確かだと思います。例えば、機械学習やディープラーニングなどの技術を使用して、より良い意思決定を行うことができます。また、自動運転車などの人工知能システムも開発されています。


In [15]:
instruction = "横須賀はどんなところ？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


横須賀市は神奈川県三浦半島にある都市で、東京湾の入り口に位置しています。


In [37]:
instruction = "横須賀はどんなところ？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


横須賀市は神奈川県南部に位置し、東京湾の入り口にあたる場所にあります。市内には約120万人の人々が暮らしており、人口密度は全国第3位となっています。また、日本有数の米軍基地である横須賀基地は、日本の国防の最重要拠点であり、国内最大の軍事都市でもあります。


In [38]:
instruction = "今のアメリカの大統領は誰ですか？"
input_text = NO_INPUT_PROMPT.format(instruction=instruction)
token_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        min_length=32,
        max_length=128,
        temperature=0.1,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_words_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(postprocess_output(output).split('応答:')[-1].strip())

Using bos_token, but it is not set yet.


ジョン・F・ケネディ大統領です！
